In [1]:
import pandas as pd
import numpy as np
import altair as alt

In [168]:
df = pd.read_csv(
    'data_individual_challenge.csv',
    sep = ';')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1035 entries, 0 to 1034
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Date        1035 non-null   object 
 1   Open        1035 non-null   float64
 2   High        1035 non-null   float64
 3   Low         1035 non-null   float64
 4   Close       1035 non-null   float64
 5   rsi         1021 non-null   float64
 6   ema_fast    1022 non-null   float64
 7   ema_slow    994 non-null    float64
 8   volatility  1021 non-null   float64
dtypes: float64(8), object(1)
memory usage: 72.9+ KB


#### Changing data types

In [54]:
df['Date'] = pd.to_datetime(df['Date'])
#df.set_index("Date", inplace = True)

#### Analyzing the big picture of he data
* std is too big in Open, High, Low and Close, there are maybe somes atyplical numbers
* rsi 70-100:sobre compra, sobre venta 0-30, nuetal 70 30

In [4]:
df.describe().round(2)

,Open,High,Low,Close,rsi,ema_fast,ema_slow,volatility
count,1035.00,1035.00,1035.00,1035.00,1021.00,1022.00,994.00,1021.00
mean,54.80,62.78,76.33,59.34,51.26,0.77,0.77,2.16
std,193.43,209.80,226.99,204.31,10.27,0.10,0.09,1.48
min,0.58,0.59,0.01,0.58,19.17,0.60,0.61,1.01
25%,0.68,0.69,0.68,0.68,44.38,0.68,0.68,1.47
50%,0.80,0.81,0.80,0.80,50.74,0.79,0.78,1.84
75%,0.87,0.88,0.87,0.87,57.69,0.86,0.86,2.24
max,892.00,934.00,899.00,921.00,79.52,0.90,0.89,17.42


In [169]:
alt.Chart(df).mark_bar().encode(
    x = alt.X(
        'volatility:Q',
        axis = alt.Axis(
            title = None),
        bin = alt.Bin(
            maxbins = 30)
    ),
    y ='count()'
)

alt.Chart(...)

* Detecting outliders

In [144]:
open_hits = alt.Chart(df).mark_bar().encode(
    x = alt.X(
        'Open:Q',
        axis = alt.Axis(
            title = None),
        bin = alt.Bin(
            maxbins = 10)
    ),
    y = alt.Y(
        'count()',
        axis = alt.Axis(
            title = None)
    )
).properties(
    width = 300,
    height = 100,
    title = 'Open Histogram'
)

close_hist = alt.Chart(df).mark_bar().encode(
    x = alt.X(
        'Close:Q',
        axis = alt.Axis(
            title = None),
        bin = alt.Bin(
            maxbins = 10)
    ),
    y = alt.Y(
        'count()',
        axis = alt.Axis(
            title = None)
    )
).properties(
    width = 300,
    height = 100,
    title = 'Close Histogram'
)

high_hist = alt.Chart(df).mark_bar().encode(
    x = alt.X(
        'High:Q',
        axis = alt.Axis(
            title = None),
        bin = alt.Bin(
            maxbins = 10)
    ),
    y = alt.Y(
        'count()',
        axis = alt.Axis(
            title = None)
    )
).properties(
    width = 300,
    height = 100,
    title = 'High Histogram'
)

low_hist = alt.Chart(df).mark_bar().encode(
    x = alt.X(
        'Low:Q',
        axis = alt.Axis(
            title = None),
        bin = alt.Bin(
            maxbins = 10)
    ),
    y = alt.Y(
        'count()',
        axis = alt.Axis(
            title = None)
    )
).properties(
    width = 300,
    height = 100,
    title = 'Low Histogram'
)

alt.hconcat(
    open_hits,
    close_hist
)& alt.hconcat(
    high_hist,
    low_hist
)

alt.VConcatChart(...)

* Removing Ouliders

In [170]:
df = df.query(
    'Open <= 100 and Close <= 100 and High <= 100 and Low <= 100'
)

* Es necesario seguir limpiando High y Low por valores atipicos poco frecuentes

In [79]:
high_time = alt.Chart(df).mark_line().encode(
    x = alt.X(
        'Date:T'
    ),
    y = alt.Y(
        'High:Q'
    )
).properties(
    width = 300,
    height = 150,
    title = 'High prince Euro:Lib'
)

low_time = alt.Chart(df).mark_line().encode(
    x = alt.X(
        'Date:T'
    ),
    y = alt.Y(
        'Low:Q'
    )
).properties(
    width = 300,
    height = 150,
    title = 'Low prince Euro:Lib'
)
alt.hconcat(high_time, low_time)

alt.HConcatChart(...)

* Second Cleaning fot Low and High

In [171]:
df = df.query(
    'High <= 1.5 and Low >= 0.5'
)

##### Checking Missing Values
* No hay muchos missing values, en rsi ema_fast and volatilyti represents 1%
* ema_slow has 3% missing values

In [114]:
df.isna().sum()

Date           0
Open           0
High           0
Low            0
Close          0
rsi            6
ema_fast       5
ema_slow      25
volatility     6
open_close     0
high_low       0
dtype: int64

* This missing data is not big, but i dont know why it doesnt exist, so i'm going to check the dates os these missing values
* All missing values are from 2000, this could be an error on api

In [112]:
df[df['rsi'].isna()]

,Date,Open,High,Low,Close,rsi,ema_fast,ema_slow,volatility,open_close,high_low
5,2000-02-07,0.6166,0.6209,0.6102,0.6192,NaN,NaN,NaN,NaN,-0.0026,0.0107
6,2000-02-14,0.6192,0.6232,0.6117,0.6157,NaN,NaN,NaN,NaN,0.0035,0.0115
7,2000-02-21,0.6161,0.6265,0.6100,0.6114,NaN,NaN,NaN,NaN,0.0047,0.0165
10,2000-03-13,0.6103,0.6186,0.6094,0.6174,NaN,NaN,NaN,NaN,-0.0071,0.0092
12,2000-03-27,0.6133,0.6141,0.5949,0.6008,NaN,NaN,NaN,NaN,0.0125,0.0192
13,2000-04-03,0.6009,0.6105,0.5958,0.6036,NaN,0.612286,NaN,NaN,-0.0027,0.0147


In [115]:
df[df['ema_slow'].isna()]

,Date,Open,High,Low,Close,rsi,ema_fast,ema_slow,volatility,open_close,high_low
5,2000-02-07,0.6166,0.62090,0.6102,0.6192,NaN,NaN,NaN,NaN,-0.0026,0.01070
6,2000-02-14,0.6192,0.62320,0.6117,0.6157,NaN,NaN,NaN,NaN,0.0035,0.01150
7,2000-02-21,0.6161,0.62650,0.6100,0.6114,NaN,NaN,NaN,NaN,0.0047,0.01650
10,2000-03-13,0.6103,0.61860,0.6094,0.6174,NaN,NaN,NaN,NaN,-0.0071,0.00920
12,2000-03-27,0.6133,0.61410,0.5949,0.6008,NaN,NaN,NaN,NaN,0.0125,0.01920
13,2000-04-03,0.6009,0.61050,0.5958,0.6036,NaN,0.612286,NaN,NaN,-0.0027,0.01470
14,2000-04-10,0.6037,0.60950,0.5986,0.6072,38.713318,0.611608,NaN,2.472826,-0.0035,0.01090
15,2000-04-17,0.6071,0.60830,0.5912,0.5938,33.291026,0.609233,NaN,2.553710,0.0133,0.01710
16,2000-04-24,0.5944,0.59770,0.5743,0.5863,30.699283,0.606176,NaN,2.686717,0.0081,0.02340
17,2000-05-01,0.5869,0.59210,0.5679,0.5876,31.691950,0.603699,NaN,2.783464,-0.0007,0.02420


#### Creating new columns
* In orden to see the diference beetween Open and Close and Low and High

In [83]:
df['open_close'] = df['Open'] - df['Close']
df['high_low'] = df['High'] - df['Low']
df.describe()

C:\Users\Anna\AppData\Local\Temp/ipykernel_2704/3897089823.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['open_close'] = df['Open'] - df['Close']
C:\Users\Anna\AppData\Local\Temp/ipykernel_2704/3897089823.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['high_low'] = df['High'] - df['Low']


,Open,High,Low,Close,rsi,ema_fast,ema_slow,volatility,open_close,high_low
count,734.000000,734.000000,734.000000,734.000000,728.000000,729.000000,709.000000,728.000000,734.000000,734.000000
mean,0.773574,0.780656,0.767061,0.773912,50.984657,0.773261,0.774469,2.037362,-0.000337,0.013595
std,0.098614,0.099525,0.097535,0.098518,10.127861,0.096471,0.092000,1.278797,0.009322,0.007611
min,0.578600,0.587600,0.567900,0.578400,19.171099,0.599133,0.607362,1.019092,-0.049600,0.002900
25%,0.682438,0.686662,0.678225,0.682600,43.985678,0.683117,0.683615,1.439539,-0.005407,0.008633
50%,0.795950,0.802405,0.790280,0.797120,50.404997,0.797290,0.810679,1.763432,-0.000425,0.012000
75%,0.863997,0.870483,0.857070,0.863762,57.513051,0.863753,0.856923,2.194614,0.004900,0.016100
max,0.961400,0.979700,0.943800,0.961600,79.522395,0.903230,0.889572,16.200247,0.070700,0.079100


#### Analysis
* Let's see the behavior in time from Open, Close, High and Low

In [124]:
open_time = alt.Chart(df).mark_line().encode(
    x = alt.X(
        'Date:T',
        axis = alt.Axis(title = None)
    ),
    y = alt.Y(
        'Open:Q',
        scale = alt.Scale(domain=[0.55,1]),
        axis = alt.Axis(title = None)
    )
).properties(
    width = 350,
    height = 150,
    title = 'Open prince Euro:Lib'
)

close_time = alt.Chart(df).mark_line().encode(
    x = alt.X(
        'Date:T',
        axis = alt.Axis(title = None)
    ),
    y = alt.Y(
        'Close:Q',
        scale = alt.Scale(domain=[0.55,1]),
        axis = alt.Axis(title = None)
    )
).properties(
    width = 350,
    height = 150,
    title = 'Close prince Euro:Lib'
)

high_time = alt.Chart(df).mark_line().encode(
    x = alt.X(
        'Date:T',
        axis = alt.Axis(title = None)
    ),
    y = alt.Y(
        'High:Q',
        scale = alt.Scale(domain=[0.55,1]),
        axis = alt.Axis(title = None)
    )
).properties(
    width = 350,
    height = 150,
    title = 'High prince Euro:Lib'
)

low_time = alt.Chart(df).mark_line().encode(
    x = alt.X(
        'Date:T',
        axis = alt.Axis(title = None)
    ),
    y = alt.Y(
        'Low:Q',
        scale = alt.Scale(domain=[0.55,1]),
        axis = alt.Axis(title = None)
    )
).properties(
    width = 350,
    height = 150,
    title = 'low prince Euro:Lib'
)

alt.hconcat(
    open_time,
    close_time
) & alt.hconcat(
    high_time,
    low_time
)

alt.VConcatChart(...)

* Let's see a candle stick using the 4 variables, also we can see the date of Boom 2008 and Brexit referendum

In [142]:
# create color conditions
color_conditions = alt.condition(
    'datum.Open <= datum.Close',
    alt.value('green'),
    alt.value('red')
)
# build chart
chart = alt.Chart(df).encode(
        x = alt.X(
            'Date',
            axis = alt.Axis(
            title = None))
        )
# set title
chart.title = 'Candle-stick Euro:Libra'
# construct a rule mark using mark_rule() method
rules = chart.mark_rule().encode(
    y =alt.Y(
        'Low',
        axis = alt.Axis(
            title = None),
        scale = alt.Scale(
            domain=[0.55,1])
    ),
    y2 = 'High'
)
# construct bars
bars = chart.mark_bar().encode(
    y = alt.Y('Open'),
    y2 = alt.Y2('Close'),
    color = color_conditions
)
# Imortant time in history
# Boom crediticio crisis 2008
boom_cred = alt.Chart(
    pd.DataFrame(
        {'Date': ['2008-09-01']})
    ).mark_rule().encode(x='Date:T')
# referendum Brexit
ref_brexit = alt.Chart(
    pd.DataFrame(
        {'Date': ['2016-06-01']})
    ).mark_rule().encode(x='Date:T')

(rules
+ bars
+ boom_cred
+ ref_brexit).properties(
    width = 700,
    height = 300
)

alt.LayerChart(...)

* The difference used to be between -0.02 and 0.02
* The difference between Open and Close shows tha big change on 2008 because for the international crisis
* 

In [148]:
open_close = alt.Chart(df).mark_line(color="#91004b").encode(
    x = alt.X(
        'Date:T'
    ),
    y = alt.Y(
        'open_close:Q'
    )
)
menos_2 = alt.Chart(pd.DataFrame({'y': [0.02]})).mark_rule().encode(y='y')
mas_2 = alt.Chart(pd.DataFrame({'y': [-0.02]})).mark_rule().encode(y='y')

(open_close 
+ menos_2 
+ mas_2
).properties(
    width = 600,
    height = 300,
    title = 'Diference Between Close and Open Euro'
)


alt.LayerChart(...)

* Oscilation of the euro:libra, how high and how low 

In [149]:
alt.Chart(df).mark_line(color="#91004b").encode(
    x = alt.X(
        'Date:T'
    ),
    y = alt.Y(
        'high_low:Q'
    )
)

alt.Chart(...)

In [30]:
df[df['Open']  <= df['Open'].quantile(0.95)]

alt.Chart(df).mark_line().encode(
    x = alt.X(
        'Date:T',
        scale = alt.Scale(nice={'interval': 'day', 'step':7})
    ),
    y = alt.Y(
        'Open:Q'
    ),
    tooltip = alt.Tooltip('month(Date)')
).properties(
    width = 220,
    height = 200
).facet(
    'year(Date)',
    columns = 4
).resolve_scale(
    x = 'independent',
    y = 'independent'
)

,Date,Open,High,Low,Close,rsi,ema_fast,ema_slow,volatility
0,2000-01-03,0.62810,633.00000,0.62350,0.62720,NaN,NaN,NaN,NaN
1,2000-01-10,0.62710,629.00000,0.61810,0.61970,NaN,NaN,NaN,NaN
2,2000-01-17,0.62000,0.62250,0.60720,609.00000,NaN,NaN,NaN,NaN
3,2000-01-24,0.60910,0.61271,599.00000,0.60270,NaN,NaN,NaN,NaN
4,2000-01-31,0.60210,0.62670,598.00000,0.61580,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1030,2019-09-30,0.89002,0.89365,0.88333,0.88779,47.151897,0.895761,0.889108,1.564625
1031,2019-10-07,0.89050,0.90183,0.86835,0.86835,37.551961,0.892106,0.888143,1.760791
1032,2019-10-14,0.87443,0.88081,0.85808,0.86451,35.993098,0.888426,0.887044,1.830085
1033,2019-10-21,0.86453,0.86687,0.85770,0.86380,35.698024,0.885143,0.885962,1.776589


* rsi shows the oscilation in 14 months before, as we can se the lowest point it takes place in 2015
* This means that euro y going to be strong in the nexts months (during the brexit debate)

In [81]:

rsi = alt.Chart(df).mark_line().encode(
    x = alt.X('Date:T'),
    y = alt.Y('rsi'),
)
line_30 = alt.Chart(pd.DataFrame({'y': [30]})).mark_rule().encode(y='y')
line_70 = alt.Chart(pd.DataFrame({'y': [70]})).mark_rule().encode(y='y')
line_100 = alt.Chart(pd.DataFrame({'y': [100]})).mark_rule().encode(y='y')

rsi+line_100+line_30+line_70

alt.LayerChart(...)

In [160]:
fast = alt.Chart(df).mark_line().encode(
    x = alt.X('Date:T',
        axis = alt.Axis(
        title = None)),
    y = alt.Y(
        'ema_fast',
        scale = alt.Scale(
        domain=[0.55,1])),
)
slow = alt.Chart(df).mark_line(color = '#ffa100').encode(
    x = alt.X('Date:T'),
    y = alt.Y('ema_slow'),
)
(fast + slow).properties(
    width = 400,
    height = 200
)

alt.LayerChart(...)

In [194]:
alt.Chart(df).mark_line().encode(
    x = alt.X('Date:T',
        axis = alt.Axis(
        title = None)),
    y = alt.Y(
        'volatility')
)

alt.Chart(...)

In [175]:
up_vol = df['volatility'].mean() + 3*df['volatility'].std()
down_vol =df['volatility'].mean() - 3*df['volatility'].std()

In [187]:
up_vol

5.8737531041320175

In [186]:
down_vol

-1.7990296696849666

In [191]:
df_new = df[(df['volatility'] < up_vol) & (df['volatility'] > down_vol)]

In [192]:
df_new.describe()

,Open,High,Low,Close,rsi,ema_fast,ema_slow,volatility
count,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,695.000000,714.000000
mean,0.776188,0.783317,0.769685,0.776576,50.808212,0.774901,0.776164,1.895852
std,0.098324,0.099214,0.097210,0.098192,10.110135,0.096667,0.092095,0.652892
min,0.578600,0.587600,0.567900,0.578400,19.171099,0.599133,0.607362,1.019092
25%,0.682700,0.686750,0.678325,0.682725,43.854565,0.682926,0.684028,1.433954
50%,0.799220,0.807535,0.794130,0.800355,50.232530,0.801188,0.811863,1.757311
75%,0.864787,0.872400,0.857575,0.864675,57.160993,0.865335,0.857503,2.170925
max,0.961400,0.979700,0.943800,0.961600,79.522395,0.903230,0.889572,5.390242
